In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import re
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import gridspec
import numpy as np
from keras.utils import to_categorical
from keras.layers import Dense, Dropout
from keras.models import Sequential

In [ ]:
from keras.datasets import imdb
(training_data, training_targets), (testing_data, testing_targets) = imdb.load_data(num_words=10000)
data = np.concatenate((training_data, testing_data), axis=0)
targets = np.concatenate((training_targets, testing_targets), axis=0)

In [ ]:
print("Categories:", np.unique(targets))
print("Number of unique words:", len(np.unique(np.hstack(data))))
length = [len(i) for i in data]
print("Average Review length:", np.mean(length))
print("Standard Deviation:", round(np.std(length)))

In [ ]:
print("Label:", targets[0])
print(data[0])

In [ ]:
index = imdb.get_word_index()
print(index['good'])
reverse_index = dict([(value, key) for (key, value) in index.items()])
print(reverse_index[49])
print(reverse_index[14-3])
print([i for i in data[0]])
decoded = " ".join( [reverse_index.get(i - 3, "#") for i in data[0]] )
print(decoded)

In [ ]:
def vectorize(sequences, dimension = 10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1
    return results 

print(type(data))
data = vectorize(data)
targets = np.array(targets).astype("float16")

In [ ]:
test_x = data[:10000]
test_y = targets[:10000]
train_x = data[10000:]
train_y = targets[10000:]

In [ ]:
model = Sequential([
    Dense(64, activation = "relu", input_shape=(10000, )),
    Dropout(.4),
    Dense(64, activation = "relu"),
    Dropout(.2),
    Dense(32, activation = "relu"),
    Dropout(.1),
    Dense(1, activation = "sigmoid")])
# model.save('model.h5')

In [ ]:
model.compile(optimizer = "adam", loss = "binary_crossentropy", metrics = ["accuracy"])
results = model.fit(train_x, train_y, epochs=2, batch_size=500, validation_data = (test_x, test_y))
print(np.mean(results.history["val_accuracy"]))

In [ ]:
def draws(H): 
    loss = H.history['loss']
    val_loss = H.history['val_loss']
    acc = H.history['accuracy']
    val_acc = H.history['val_accuracy']
    epochs = range(1, len(loss) + 1)

    fig = plt.figure(figsize=(12, 6))
    gs = gridspec.GridSpec(1, 2, width_ratios=[3, 3])
    plt.subplot(gs[0])
    plt.plot(epochs, loss, 'r--', label='Training loss')
    plt.plot(epochs, val_loss, 'g--', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(gs[1])
    plt.plot(epochs, acc, 'r--', label='Training acc')
    plt.plot(epochs, val_acc, 'g--', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

draws(results)

In [ ]:
def load_text():
    dictionary = imdb.get_word_index()
    load_x = []

    words = input()
    words = re.sub(r"[^a-zA-Z0-9']", " ", words)
    words = words.split(' ')

    valid = []
    for word in words:
        word = dictionary.get(word) # в число
        if word in range(1, 10000):
            valid.append(word+3)
    load_x.append(valid)

    print(load_x)
    load_x = vectorize(load_x)
    result = model.predict(load_x)
    print(result)

load_text()    